In [1]:
import pandas as pd
import json
from pprint import pprint
import numpy as np
from os import listdir
from os.path import isfile, join

In [3]:
def get_data(filename:str):
    with open(filename) as f:
        json_file = json.load(f)
    
    data = json_file['people'][0]['pose_keypoints_2d']
    assert len(data) == 75
    return data

def pipeline(path, label):
    
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    df = pd.DataFrame(columns=list(range(75)))
    for i, file in enumerate(onlyfiles):
        df.loc[i] = get_data(path+file)
    df['label'] = label
    return df


def select_frame(df, single=True):
    # Take df where each row is a frame in the video and return
    # either 1 still frame or multiple still frames
    print(len(df))
    
    deltas = []
    for i in range(len(df)-1,0,-1):
        # iterate in reverse order as best poses often last
        current_ = df.iloc[i]
        next_ = df.iloc[i-1]
        deltas.append(np.mean(abs(next_ - current_)))
    SD = np.std(deltas)
        
    if single:
        # Single Frame
        for i in range(len(df)-1,0,-1):
            # iterate in reverse order as best poses often last
            current_ = df.iloc[i]
            next_ = df.iloc[i-1]
            d = np.mean(abs(next_ - current_)) 
            if d < SD:
                print(i)
                return df.iloc[i]
    else:
        # Multiple Frames
        ds = deltas < SD*0.5
        return df[:-1][ds]
    
def s3_get(filename, folder=''):
    import boto3
    import json

    s3 = boto3.resource('s3')

    content_object = s3.Object('alignedstorage' , folder + filename)
    file_content = content_object.get()['Body'].read().decode('utf-8')
    return file_content
    
        

good_data = pipeline('warrior/', label=1)
bad_data = pipeline('warrior_incorrect/', label=0)
#get_data('warrior/warrior_000000000039_keypoints.json', classification = 'good')
    

In [4]:
#select_frame(good_data, single=True)
s3_get('warrior2_arms_1.csv', folder='training_data')

ClientError: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied

In [25]:
full_data = pd.concat([good_data,bad_data])
full_data.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,label
0,664.680,204.282,0.918086,692.073,259.115,0.875232,646.980,257.194,0.832140,572.597,...,515.760,609.804,0.599547,521.606,603.885,0.594741,576.510,601.873,0.755656,1
1,664.665,204.246,0.918837,692.090,259.068,0.875573,646.958,257.168,0.822406,572.624,...,515.737,611.718,0.592743,521.600,603.895,0.596312,576.498,601.908,0.757170,1
2,664.684,206.193,0.931179,693.983,257.119,0.875670,647.023,255.218,0.807673,572.645,...,517.692,611.715,0.589091,521.646,603.913,0.586224,578.445,601.916,0.748488,1
3,666.623,212.076,0.964515,696.038,255.150,0.897790,649.037,253.210,0.800950,574.538,...,515.785,609.764,0.605959,521.614,603.875,0.596971,578.452,601.928,0.742447,1
4,666.646,212.079,0.957779,696.036,255.137,0.896309,649.010,251.283,0.797011,574.496,...,515.775,609.786,0.600176,521.619,603.891,0.589176,578.461,601.957,0.746312,1


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

x = full_data.drop(columns=['label'])
y = full_data['label']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy_score(y_test, preds)


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


1.0

In [ ]:
s3_resource = boto3.resource('s3')
s3_resource.Object('alignedstorage', f'{node}-{key}.csv').put(Body=csv_buffer.getvalue())